In [1]:
!wget https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
!unzip trainDevTestTrees_PTB.zip

'''
http://localhost:8888/tree
'''

--2022-01-12 15:10:42--  https://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 789539 (771K) [application/zip]
Saving to: ‘trainDevTestTrees_PTB.zip.4’

trainDevTestTrees_P 100%[===================>] 771.03K   215KB/s    in 3.6s    

2022-01-12 15:10:47 (215 KB/s) - ‘trainDevTestTrees_PTB.zip.4’ saved [789539/789539]

Archive:  trainDevTestTrees_PTB.zip
replace trees/dev.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


'\nhttp://localhost:8888/tree\n'

In [ ]:
from tqdm import tqdm

In [10]:
# convert txt to csv
import nltk
a = nltk.corpus.BracketParseCorpusReader("trees", "(train|dev|test)\.txt")

text = {}
labels = {}
keys = ['train', 'dev', 'test']
for k in keys :
    text[k] = [x.leaves() for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    labels[k] = [int(x.label()) for x in a.parsed_sents(k+'.txt') if x.label() != '2']
    print(len(text[k]))


6920
872
1821
6920
872
1821
6920
872
1821
6920
872
1821


In [12]:
# convert txt to csv
import pandas as pd
df_texts = []
df_labels = []
df_exp_split = []

for k in keys :
    df_texts += text[k]
    df_labels += labels[k]
    df_exp_split += [k]*len(text[k])
    
data = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_split})

In [13]:
import spacy
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])
import re

def tokenize(text) :
    text = " ".join(text)
    text = text.replace("-LRB-", '')
    text = text.replace("-RRB-", " ")
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    tokens = " ".join([t.text.lower() for t in nlp(text)])
    return tokens


In [14]:
for k in keys :
    text[k] = [tokenize(t) for t in text[k]]
    labels[k] = [1 if x >= 3 else 0 for x in labels[k]]

In [15]:
import os
os.makedirs('data', exist_ok=True)

In [16]:
import spacy
nlp = spacy.load('en')

In [24]:
def clean_text(text) :
    return "\n".join([" ".join([t.text.strip() for t in sent]) for sent in nlp(str(text)).sents])

tqdm.pandas()
data["text"] = data.text.progress_apply(lambda x : clean_text(x))

100%|██████████| 9613/9613 [01:32<00:00, 104.07it/s]


In [25]:
data["label_id"] = data.label.apply(lambda x : "positive" if x == 1 else "negative")

In [26]:
pd.options.mode.chained_assignment = None 
train = data.loc[data.exp_split == "train"]
train["annotation_id"] = list(range(len(train)))
train.annotation_id = train.annotation_id.apply(lambda x: "train_" + str(x))
train.to_csv("data/train.csv", index = False)

dev = data.loc[data.exp_split == "dev"]
dev["annotation_id"] = list(range(len(dev)))
dev.annotation_id = dev.annotation_id.apply(lambda x: "dev_" + str(x))
dev.to_csv("data/dev.csv", index = False)

test = data.loc[data.exp_split == "test"]
test["annotation_id"] = list(range(len(test)))
test.annotation_id = test.annotation_id.apply(lambda x: "test_" + str(x))
test.to_csv("data/test.csv", index = False)